<a href="https://colab.research.google.com/github/mscholl96/mad-recime/blob/network_LSTM/network/LSTM/instrGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instruction Generation

## Basic includes

In [ ]:
!pip install ipython-autotime
%load_ext autotime
!pip install word2vec
!pip install ray[tune]

## Connect colab and set paths

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

rootDir = '/content/drive/MyDrive/'

TIMESTAMP = '2022_03_19'

dataPath = rootDir + 'TP2/Datasets/Recipe1M/' + TIMESTAMP
tarPath = rootDir + 'Colab Notebooks/recime/data/' + TIMESTAMP


import sys
sys.path.append(rootDir + 'Colab Notebooks/recime/LSTM')


In [1]:
# local exec
tarPath = 'D:/02_Studium/SBX/mad-recime/data/2022_03_19'

import os
sys.path.append(os.getcwd())

## Basic includes

In [2]:
import pandas as pd
import numpy as np

import torch

# Model
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable 
from torchsummary import summary

# Optimizer
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split


# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

# hyperparameter tuning
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

## Seed

In [3]:
torch.manual_seed(0)
np.random.seed(0)

## Training execution
mixture of 
* https://pytorch.org/tutorials/beginner/introyt/trainingyt.html
* https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html
* https://stackoverflow.com/questions/67295494/correct-validation-loss-in-pytorch


### Set device

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == "cuda":
    print(torch.cuda.get_device_name(0))

### Set hyperparams

In [ ]:
from src.instGen import HyperParams

hyperParams = HyperParams(epochs=50, batchSize=32)
print(hyperParams)

### Set dataset

In [ ]:
from src.instGen import InstructionSet
from src.preProc import getPreProcData

instSet = InstructionSet(tarPath, setSize=2)

# split data set
trainNum = int(hyperParams.ratio[0] * len(instSet))
valNum = int(hyperParams.ratio[1] * len(instSet))


splitSet = random_split(
    instSet, [trainNum, valNum], generator=torch.Generator().manual_seed(0))
    
splitSet = {'train': splitSet[0], 'val': splitSet[1]}

testSet = getPreProcData(tarPath, range(-1,0))

In [ ]:
pd.DataFrame.from_dict(pd.Series(instSet.tokenizer.word_index))

### Set model

In [ ]:
from src.instGen import Model3

model = Model3(hyperParams, instSet, device)
model.to(device)
print(model)


### Train

In [ ]:
from src.instGen import train

train(splitSet, model, hyperParams, device)

### Test

In [ ]:
from src.instGen import predict, sample

# sample(model, titleSet, 6, device, initial=['dry', 'penne', 'pasta', 'broccoli', 'sun', 'dried', 'tomatoes', 'packed', 'in', 'oil', 'garlic', 'cloves', 'cheddar', 'cheese', 'salt', 'black', 'pepper'])
seq = splitSet['test'][np.random.randint(0, len(splitSet['test']))][0].tolist()

def remove_values_from_list(the_list, val):
   return [instSet.t300kenizer.index_word[value] for value in the_list if value != val]

seq = remove_values_from_list(seq, 0)
print(seq)

sample(model, instSet, 300, device, initial=seq)

## Save model

In [ ]:
torch.save(model.state_dict(),
           rootDir + 'weights/instGenerator_model.pt')


# Tensorboard visualization

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=/content/drive/MyDrive/runs/instTrainer